# Imports and Function Definitions

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
#Function for quantifying test line intensity
def testIntensity(src):
    margin = 0 #margin around image, not relevant when doing manual cropping
    step = 1 #Step size when searching for test line
    controlToTest = 450 #Initial guess at distance between test and control lines
    scan = 20 #size of area in pixels that is searched for location of test line
    refScan = 10 #size of reference area in pixels
    lineWidth = 6 #Half-width of line
    src = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY) #convert image to grayscale
    img = cv2.rotate(src, cv2.ROTATE_90_COUNTERCLOCKWISE) #rotate image
    
    h,w = img.shape
    
    img = img[margin:h-margin, margin:w-margin]
    
    h,w = img.shape #get height and width
    
    #Convert image to 1D array of pixel intensities. For each row, use median value of grayscale pixel intensity
    mean = np.median(img,axis=0)
    mean = 255 - mean #so that high values correspond to dark areas
    
    #Looking for control line
    
    #Left/right bounds on area where control line can be found
    left=0
    right=int(w/2)
    
    LFAControl = mean[left:right] #cropped section of area where control line could be
    ControlLineLoc = np.argmax(LFAControl) # potential location of control line
    
    #make sure control line location isn't the leftmost part of the image (this would be inaccurately focusing on a shadow)
    while(ControlLineLoc == left):
        left += step
        LFAControl = mean[left:right]
        ControlLineLoc = left + np.argmax(LFAControl)
    
    #make sure control line location isn't the rightmost part of the area (this is probably the test line)
    while(ControlLineLoc == right):
        right -= step
        LFAControl = mean[left:right]
        ControlLineLoc = left + np.argmax(LFAControl)
    
    #Search for test line
    TestLineLoc = ControlLineLoc + controlToTest
    
    if TestLineLoc > len(mean):
        TestLineLoc = ControlLineLoc + 75
    if TestLineLoc + scan > len(mean):
        scan = len(mean)-TestLineLoc
    
    TestLineLocation = TestLineLoc-scan + np.argmax(mean[TestLineLoc-scan:TestLineLoc+scan])
    controlToTest = TestLineLocation - ControlLineLoc
    
    #Determine reference value (value of background intensity)
    ReferenceLocation = int((ControlLineLoc + TestLineLocation)/2)
    Reference = np.mean(mean[ReferenceLocation-refScan:ReferenceLocation+refScan])
    
    #subtract reference value from entire array, and make sure negative values are converted to 0
    LFACorrected = mean-Reference
    LFACorrected[LFACorrected < 0] = 0
    
    #Determine line intensities by taking sum of corrected intensity, centered at each line location
    ControlLine = np.sum(LFACorrected[ControlLineLoc-lineWidth:ControlLineLoc+lineWidth])
    
    TestLine = np.sum(LFACorrected[TestLineLocation-lineWidth:TestLineLocation+lineWidth])
    
    #return values
    return controlToTest,Reference,ControlLine,TestLine,TestLine/ControlLine

# Paths for input and output

In [ ]:
input_dir = r"" #folder containing cropped images of detection zones
output_path = r"" #excel file that will be output to

# Generating Excel file for output

In [ ]:
df = pd.DataFrame(columns=('ControlToTest','Reference','Control Integral','Test Integral', 'Normalized Test Line'))

for filename in os.listdir(input_dir):
    f = os.path.join(input_dir, filename)
    img = cv2.imread(f)
    df.loc[filename] = testIntensity(img)
    
df.to_excel(output_path) 